**<p style = "text-align: center">TRƯỜNG ĐẠI HỌC KHOA HỌC TỰ NHIÊN - ĐẠI HỌC QUỐC GIA TPHCM</p>**

# **<p style = "text-align: center">NHẬP MÔN KHOA HỌC DỮ LIỆU</p>**
## **<p style = "text-align: center">Đồ án Cuối kì</p>**
### <p style = "text-align: center"> <b>Đề tài:</b> Phân tích, khám phá dữ liệu về sách trên web thương mại điện tử (tiki)</p>
## **<p style = "text-align: center">THU THẬP DỮ LIỆU</p>**
**Sinh viên thực hiện**

| Mã số sinh viên | Họ và tên |
| --- | --- |
| 20120040 | Nguyễn Quang Gia Bảo |
| 20120136 | Huỳnh Tuấn Nam |
| 20120158 | Trần Hoàng Anh Phi |

#### **Install các thư viện cần thiết**

In [ ]:
# !pip install httpx

#### **Import packages**

In [ ]:
import json
import pandas as pd
# import requests
import json
import httpx
import urllib.request

Dùng scrapy
-> Gặp lỗi 403 forbidden và không get được dữ liệu

-> Selenium cũng tương tự, javascript chỉ load sau khi đã raise exception timeout

## **Crawl tiki**

Lấy danh mục sản phẩm trong danh mục sách tiếng Việt ở api của tiki

In [ ]:
def generate_url(category, page, urlKey):
    return f'https://tiki.vn/api/personalish/v1/blocks/listings?limit=100&aggregations=2&sort=top_seller&category={category}&page={page}&urlKey={urlKey}'


page = httpx.get(generate_url('316', '1', 'sach_truyen_tieng_viet')).json()
for i in page['filters']:
    if i['display_name'] == 'Danh Mục Sản Phẩm':
        categories = i['values']
        break

print(categories[0])        
    
all_categories = []
for cat in categories:
    cat_data = httpx.get(generate_url( cat['query_value'], '1', cat['url_key'] )).json()
    has_child = False
    for i in cat_data['filters']: 
        if i['display_name'] == 'Danh Mục Sản Phẩm': # Nếu chứa danh mục con thì thêm vào
            #print(i['display_name'])
            all_categories.extend(i['values'])
            has_child = True
    
    if not has_child: 
        all_categories.append(cat)
        
print(len(all_categories))


In [ ]:
categories[0]

api limit = 100 -> Mỗi lần request lấy được 100 giá trị</br>
Cần lấy giá trị tối đa của số sách có thể có trong một thể loại 

**Lấy id của các cuốn sách ở từng thể loại**

In [ ]:
# all_categories = pd.DataFrame(all_categories)
# display(all_categories)
id_df = []
def get_books_id(cat):
    n = (cat['count']//100) + 2
    books = {
        'categories': cat['display_value'],
        'books_id': []
    }
    for i in range(1, n):
        response = httpx.get(generate_url(cat['query_value'], i, cat['url_key'] )).json()
        if 'data' in response:
            for book in response['data']:
                books['books_id'].append({'id': book['id']})
                id_df.append({'id': book['id']})
    
    return books


In [ ]:
books_id_all_cate = []
for cate in all_categories:
    print(cate['display_value'])
    books_id_all_cate.append(get_books_id(cate))

In [ ]:
categories_id = pd.DataFrame(books_id_all_cate)
categories_id.to_csv('id_data/categories_id.csv', encoding='utf-8-sig')
id_df = pd.DataFrame(id_df)
with open('id_data/id_df.json', 'w', encoding='utf-8-sig') as file:
    id_df.to_json('id_data/id_df.json', force_ascii=False, orient='records')
    

## **Lấy thông tin của từng cuốn sách dựa vào id đã thu thập**

Đọc id từ file id đã lấy được

In [ ]:
try:
    with open('id_data/id_df.json') as f:
        id = json.load(f)
except IOError:
    print("File not found")
    
books_id = [i.get('id') for i in id]


In [ ]:
len(books_id)

Viết các function để lấy dữ liệu

In [ ]:

def get_book_info(raw_book):
    book_info = {
                'id': raw_book.get('id'), 
                'master_id': raw_book.get('master_id'),
                'sku': raw_book.get('sku'),
                'name': raw_book.get('name'),
                'short_url': raw_book.get('short_url'),
                #'short_description': raw_book.get('short_description'),
                'book_cover': raw_book.get('book_cover'),
                'price': raw_book.get('price'),
                'original_price': raw_book.get('original_price'),
                'discount_rate': raw_book.get('discount_rate'),
                'rating_average': raw_book.get('rating_average'),
                'review_count': raw_book.get('review_count'),
                'inventory_type': raw_book.get('inventory_type'),
                'productset_group_name': raw_book.get('productset_group_name'),
                'day_ago_created': raw_book.get('day_ago_created'),
                'categories': raw_book.get('categories').get('name') if raw_book.get('categories') not in [None,[]] else None,
                'all_time_quantity_sold': raw_book.get('all_time_quantity_sold', 0)
                }
    
    # authors_name = []
    # authors_name = []
    if raw_book.get('authors') not in [None, []] :
        authors_name = [i.get('name') for i in raw_book.get('authors')]
        book_info['authors'] = ', '.join(authors_name)
    else: book_info['authors'] = ''

    if raw_book.get('specifications') not in [None, []]:
        # print(raw_book['specifications']==[])
        for i in raw_book['specifications'][0]['attributes']:
            # print(i)
            book_info[i['code']] = i['value']
    return book_info

def split_books_id(books_id: list, i, n): # Part 0: i = 0
    length_path = len(books_id)//n
    start = length_path*i
    end = length_path+length_path*i
    if i == n - 1:
        end = len(books_id)
    return start, end

def get_all_books(book_df, start, end):
    for id in books_id[start:end]:
        url = f'https://tiki.vn/api/v2/products/{id}'
        try:
            response = httpx.get(url)
            raw_book = response.json()
            print(response.status_code)
            if response.status_code == 200:
                book_info = get_book_info(raw_book)
                book_df.append(book_info)              
        except:
            book_df.append(get_book_info(raw_book))
            print(f'ERROR:', id)
    return book_df
            


**Vì lượng dữ liệu khá lớn và khó xử lý nên ta sẽ chia thành các phần nhỏ sau đó concat lại:**

In [ ]:
# start, end = split_books_id(books_id, i=0, n=5)
# book_df1 = get_all_books(book_df, start, end)
# df1 = pd.DataFrame(book_df1)
# df1.to_csv('full_data/part_1')

Quên encoding :(

In [ ]:
df = pd.read_csv('full_data/part_1.csv', encoding='utf-8')
df.to_csv('full_data/part_1.csv', encoding='utf-8-sig', index=False)

Tiếp tục với part 2, 3, ..

In [ ]:
all_df = []
for i in range(1, 5):
    book_df = []
    start, end = split_books_id(books_id, i, n=5)
    book_df = get_all_books(book_df, start, end)
    df = pd.DataFrame(book_df)
    df.to_csv(f'full_data/part_{i+1}.csv', encoding='utf-8-sig')
    all_df.append(df)

Mỗi part mất khoảng 150 phút 😅

***

**Concat các file lại thôi**

In [ ]:
import glob
import pandas as pd

df = pd.concat(map(pd.read_csv, glob.glob('full_data/*.csv')))
df.to_csv('full_data/data.csv', index=False, encoding = 'utf-8-sig')

In [ ]:
df.shape

**Thử lại với multithread**

In [ ]:
import concurrent.futures 
import time

NUMBER_OF_WORKERS = 4
        
def handle_get_book_info(book):
    url = f'https://tiki.vn/api/v2/products/{book["id"]}'
    try:    
        response = httpx.get(url)   
        print(response.status_code)
        if response.status_code == 200: #and 'content-type' in response.headers 
            #and 'application/json' in response.headers['content-type']):
            raw_book = response.json()
            
            book_info = get_book_info(raw_book, book)
            return book_info
        # response.raise_for_status()
        
    except Exception as e:
        # df.append(book)
        # book_info = response.json()
        # if response.status_code == 429:
        #     time.sleep(int(response.headers["Retry-After"]))
        print(f'ERROR {str(e)}: ', book['id'])
        # elif response.status_code == 200:
        #     book_info = get_book_info(response.json(), book)
        # book_info = raw_book
        # return book_info
        # return response.json()
    # return book_info
        
test_df = []

with concurrent.futures.ThreadPoolExecutor(max_workers=NUMBER_OF_WORKERS) as executor:
    all_params = list(book for cate in books_id_all_cate[:1] 
                      for book in cate['Book_id'] )
    for future in concurrent.futures.as_completed(
        executor.submit(handle_get_book_info, params)
        for params in all_params
    ):
        # result = future.result()
        test_df.append(future.result())
        #print(result)

Không hiệu quả vì tạo ra 429 error -> Too many requests